# <center>Adaptive Exponential Weights Algorithm for <br/> Learning Equilibrium Flows of the Routing Game</center>

In [1]:
# Import necessary libraries
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import random
import networkx as nx   # best working with networkx 2.5 to avoid errors
from scipy import integrate
import os
current_dir = os.getcwd()
import pickle
import copy

## 0. Preliminary functions

In [2]:
# Preliminary function to import edge data:
def read_edge(network_name):
    netfile = os.path.join(root, network_name,network_name + '_net.tntp')
    net = pd.read_csv(netfile, skiprows=8, sep='\t')
    trimmed= [s.strip().lower() for s in net.columns]
    net.columns = trimmed
    # And drop the silly first andlast columns
    net.drop(['~', ';'], axis=1, inplace=True)
    return net

In [3]:
# Functions to generate random noises given a network and a time horizon:
def generate_noise(network,T,bound_noise_rel=1,StdDevNoise_abs=10):
    network_noise = nx.DiGraph(T=T)
    network_noise.add_edges_from(list(network.edges()))
    for e in network_noise.edges():
        network_noise[e[0]][e[1]]['noise_rel'] = bound_noise*np.random.uniform(-1,1,T+1)
        network_noise[e[0]][e[1]]['noise_abs'] =  np.random.normal(0,StdDevNoise_abs,T+1)
    return network_noise

In [4]:
# Compute the costs on all edges of the network (already assigned flows and exo_load) 
def compute_e_cost(network,edges):
    for e_index in range(len(edges)):        #for each edge in the network
        # Import edges data
        e_head = edges['init_node'][e_index]
        e_tail = edges['term_node'][e_index]
        free_flow = edges['free_flow_time'][e_index]
        B =  edges['b'][e_index]
        Power =  edges['power'][e_index]
        capacity =  edges['capacity'][e_index]
#         #Define the BPR function
        term = (network[e_head][e_tail]['load'] )/capacity
        e_cost = free_flow * ( 1 + B * (term**Power)) *(1+ network[e_head][e_tail]['noise_rel'])       #Multiplicative noise
        e_cost += network[e_head][e_tail]['noise_abs']
        network[e_head][e_tail]['cost'] = e_cost
    return

# Compute the costs on a path [i,p] of the network (alredy assigned flows and exo_load) 
def p_cost(network,Paths,i,p):
    path_p = Paths[i][p]
    cost = 0
    for node in range(len(path_p) -1):
        e_head=path_p[node]
        e_tail = path_p[node+1]
        if 'cost' not in network[e_head][e_tail]: network[e_head][e_tail]['cost'] =0  # In case, cost has not been generated
        cost+= network[e_head][e_tail]['cost']
    return cost

In [5]:
def load(network,flow): 
    for e in network.edges():
        edge = network[e[0]][e[1]]
        edge['load']= 0  #initialize the load on edge
        if 'passing_paths' in edge:
            for pair_path in edge['passing_paths']:
                i = pair_path[0]
                p = pair_path[1]
                edge['load']+= flow[i][p]
    return

In [6]:
### the MEAN Potential function with Beta-distribution noise
def potential(flow,pairs,network,edges, mean_noise):
    load(network,flow)
    Out = 0  #Initialize output
    for e_index in range(len(edges)):        #for each edge in the network
        # Import edges data
        e_head = edges['init_node'][e_index]
        e_tail = edges['term_node'][e_index]
        free_flow = edges['free_flow_time'][e_index]
        B =  edges['b'][e_index]
#         Power =  edges['power'][e_index]
        Power =1
        capacity =  edges['capacity'][e_index]
        if 'load' not in network[e_head][e_tail]:  network[e_head][e_tail]['load'] =  0
        if 'exo_load' not in network[e_head][e_tail]:  network[e_head][e_tail]['exo_load'] =  0
        #Define the BPR function
        def latency(x):
            return free_flow * ( 1 + B * ((x/capacity)**Power) )
        # The potential value (with mean_noise = 0):
        Out+= integrate.quad(latency, 0, network[e_head][e_tail]['load'])[0]    
    return Out

In [7]:
# Functions to store the results:
def save_output(directory, outputEW=[],outputAcceEW=[],outputAdaEW=[]):
    if len(outputEW) >0:
        with open(directory+'outputEW.data', 'wb') as filehandle:
            pickle.dump(outputEW, filehandle)   
    if len(outputAcceEW) >0:
        with open(directory+'outputAcceEW.data', 'wb') as filehandle:
            pickle.dump(outputAcceEW, filehandle)   
    if len(outputAdaEW) >0:
        with open(directory+'outputAdaEW.data', 'wb') as filehandle:
            pickle.dump(outputAdaEW, filehandle)   
def save_potential(directory,potEW=[],potAcceEW=[],potAdaEW=[]):
    if len(potEW)>0:
        dfEW = pd.DataFrame({'potEW':potEW})
        dfEW.to_csv(directory+'potEW.csv', index=False)
    if len(potAcceEW)>0:
        dfAcceEW = pd.DataFrame({'potAcceEW':potAcceEW})
        dfAcceEW.to_csv(directory+'potAcceEW.csv', index=False)
    if len(potAdaEW)>0:
        dfAdaEW = pd.DataFrame({'potAdaEW':potAdaEW})
        dfAdaEW.to_csv(directory+'potAdaEW.csv', index=False)

## 1. Import Networks Data

Now, we can input a network instance by the following syntax:

In [8]:
network_name = 'SiouxFalls'
# network_name = 'Austin'
# network_name = 'Eastern-Massachusetts'
# network_name = 'Braess-Example'

In [9]:
root = os.path.dirname(os.path.abspath('.'))     #Look for the root folder
root = root+'\\TransportationNetworks-master'
directory = current_dir + "/data/"+ network_name + '/'
os.makedirs(os.path.dirname(directory), exist_ok=True)
directory_sol =  current_dir + "/solutions/"+ network_name + '/'
os.makedirs(os.path.dirname(directory_sol), exist_ok=True)

2) We choose an instance (generated via Passing_data_to_gpickle.ipynb):

In [10]:
N = 5 ## Choose the number of pairs:
print('Number of pairs = ', N)
## Load the network:
network= nx.read_gpickle(directory +"/graph,N="+str(N)+ ".gpickle")      
print(nx.info(network))                               
pairs = pd.read_csv(directory+'pairs,N='+str(N)+'.csv')

edges = read_edge(network_name) ## Load the edges data:
## Load the path data:
with open(directory+'paths,N='+str(N)+'.data', 'rb') as filehandle:
    paths = pickle.load(filehandle) 
Num_p = 0
for i in range(N):
    Num_p+= len(paths[i])
print('Number of paths = ', Num_p) 

Number of pairs =  5
Name: 
Type: DiGraph
Number of nodes: 24
Number of edges: 76
Average in degree:   3.1667
Average out degree:   3.1667
Number of paths =  50


# 3. Exponential Weight Algorithm


The ouput of this algorithm is a class, called ```solution```, where:
* ```solution.pot``` is a $T_\textrm{term}$ dimensional list of values of $\Phi(F[t])$ for $t=1,...,T_\textrm{term}$; where $T_\textrm{term}$ is the time the algorithm terminates. 
* ```solution.flag``` shows the termination condition/ error flag. The list of flags are: ```'Normal execute'```, ```'Terminate due to "output_t = NaN"'``` (if this is the case, should reduce stepsize).

In [11]:
def EW(pairs,network,paths,edges,T,network_noise,mean_noise,stoch = True, fixed_stepsize=False, stepsize=1, save_output = False):     #Include the input paths (pre-computed to save computations)
    print('#----------------------------------','\n','Run EW algorithm with T = ', T, ', stochastic = ', stoch)
    # Checking conditions of inputs:
    if not (len(pairs)== len(paths)):
        print('Error: inconsistent dimensions of pairs and paths')
        return
    if (stoch==True ) and ((network.number_of_edges()!= network_noise.number_of_edges()) or (T != network_noise.graph['T'])):
        print('Error: inconsistent data in network_noise')
        return    
    N=len(pairs)   
    #INITIALIZATIONS
    flag = 'Normal execute'
    error = 0 
    pot = []
    outputXt = []
    output= []
    ## Create for a zeros vector
    zeros=[]
    for i in range(N):
        zeros.append(np.zeros(len(paths[i])))
    ### Initialize Y at zeros
    Y = copy.deepcopy(zeros)
    X_cumul = copy.deepcopy(zeros)
    # Reset the noise on each edge:    
    for e_index in range(len(edges)):
        e_head = edges['init_node'][e_index]
        e_tail = edges['term_node'][e_index]
        network[e_head][e_tail]['noise_rel']=0
        network[e_head][e_tail]['noise_abs']=0
        network[e_head][e_tail]['exo_load']=0
        
                    #-----------------------------------------------------------#
                    #                  START OF MAIN ALGORITHM                  #
                    #-----------------------------------------------------------#
    for t in range(1,T+1):
        if (t % 500 ==1): print('t=',t)    #To check the algorithm is running
        if (t==T): print('T_terminal=',t)
        #-----------------------------------------------
        #     Start computing Xt (the output at time t)
        #-----------------------------------------------
        Xt=[]
        X_avg=[]
        for i in range(N):
            Xti,X_avg_i = [],[]
            for p in range(len(paths[i])):
#                 if t==1: print('This is Y[i] = ', np.array(Y[i]))
                Xtip = float(pairs['demand'][i]) / float(np.sum( np.exp(np.array(Y[i]) - float(Y[i][p]))))
                Xti.append(Xtip)
                X_cumul[i][p] += Xtip
                X_avg_i.append(float(X_cumul[i][p])/t)
            #Finish computing Xi
            if np.isnan(np.sum(Xti)):   #IF there exists an NaN in Xi, then terminate
                flag = 'Terminate due to "output_t = NaN"'
                print('T_terminate=',t)
                error =1
                break
            Xt.append(Xti)
            X_avg.append(X_avg_i)
        if (error ==1): break     #If there exist NaN in Xi; then break and terminate
        # Finish computing and store X_avg
        if save_output==True: 
            output.append(X_avg)
            outputXt.append(Xt)
        
        
      
        #----------------------------------------------
        #     Generate exogeneous_load if stoch= True
        #---------------------------------------------
        if stoch == True:
            for e_index in range(len(edges)):
                e_head = edges['init_node'][e_index]
                e_tail = edges['term_node'][e_index]
                network[e_head][e_tail]['noise_rel']= network_noise[e_head][e_tail]['noise_rel'][t]    #load the exo_load from network_noise to network
                network[e_head][e_tail]['noise_abs']= network_noise[e_head][e_tail]['noise_abs'][t]    #load the exo_load from network_noise to network
        
        #--------------------------------------------------------------
        #     Compute potential value for storing purposes
        #---------------------------------------------------------------
        load(network,X_avg)                       # Import load at X_avg to network         
        pot.append( potential(X_avg,pairs,network,edges, mean_noise)) # Store the potential of X_avg
        #IF we want to compare performance of Xt
#         potXt.append(potential(Xt,pairs,network,edges)) # Store the potential of X_avg
    
        #----------------------------------------------------
        #      Updating the weight Y
        #---------------------------------------------------
        load(network,Xt)                       # Import load at Xt to network to compute cost        
        compute_e_cost(network,edges)             #Generate the corresponding costs on all edges:
        if fixed_stepsize == False:
            for i in range(N):
                for p in range(len(paths[i])):
                    Y[i][p] = Y[i][p] - stepsize* (1/np.sqrt(t)) * p_cost(network,paths,i,p)
        else: 
            for i in range(N):
                for p in range(len(paths[i])):
                    Y[i][p] = Y[i][p] - stepsize * p_cost(network,paths,i,p)
    
    class solution:
        def __init__(self,pot,output,outputXt,flag):
            self.output = output
            self.outputXt = outputXt
            self.pot = pot
#             self.potXt = potXt
            self.flag= flag        
    return solution(pot,output,outputXt,flag)

    


## 4. Accelerated Exponential Weights Algorithm

* ```beta``` = a constant which should be chosen as an upper-bound of the smoothness level of the latency functions

In [12]:
def AcceEW(pairs,network,paths,edges,T,network_noise,mean_noise,beta,stoch,save_output=False):    
    print('#----------------------------------','\n','Run AcceWeight algorithm with T = ', T, ', stochastic = ', stoch)
    # Checking conditions of inputs:
    if not (len(pairs)== len(paths)):
        print('Error: inconsistent dimensions of pairs and paths')
        return
    if (stoch==True ) and ((network.number_of_edges()!= network_noise.number_of_edges()) or (T != network_noise.graph['T'])):
        print('Error: inconsistent data in network_noise')
        return     
    N=len(pairs)
    #INITIALIZATIONS
    pot = []
    output = []
    flag = 'Normal execute'
    error = 0
    #Initialize stepsizes:
    alpha = 0
    gamma0 = 1/(N* max(pairs['demand']) *beta)
    gamma = gamma0
    # Initialization for Xt, Y, Z_tilde,Zt
    zeros=[]
    for i in range(N):
        zeros.append(np.zeros(len(paths[i])))
    ### Initialize Y,Zt,Z_tilde at zeros
    Y = copy.deepcopy(zeros)
    Zt= copy.deepcopy(zeros)
    Xt= copy.deepcopy(zeros)
  
    for e_index in range(len(edges)):
        e_head = edges['init_node'][e_index]
        e_tail = edges['term_node'][e_index]
        network[e_head][e_tail]['noise_rel']=0
        network[e_head][e_tail]['noise_abs']=0
        network[e_head][e_tail]['exo_load']=0
        
                            #-----------------------------------------------------#
                            #              START OF MAIN ALGORITHM                #
                            #-----------------------------------------------------#
    for t in range(1,T+1):
        if (t % 500 ==1): print('t=',t)    #To check the algorithm is running
        if (t==T): print('T_terminal=',t)        
        #----------------------------------------#
        #         Compute Zt                     #
        #----------------------------------------#       
        X_prev = copy.deepcopy(Xt)
        Xt=[]
        for i in range(N):
            Xt_i=[]
            for p in range(len(paths[i])):
                diff = np.sum(np.exp(np.array(Y[i]) - Y[i][p]))
                Zt[i][p] = float(pairs['demand'][i])/  diff
                Xt_ip = alpha*X_prev[i][p]  + (1-alpha)* Zt[i][p]
                Xt_i.append(Xt_ip)     #Compute X_ip and append it into Xi array
            # Checking error in computing Zi
            if np.isnan(np.sum(Zt[i])):   #IF there exists an NaN in Fi, then terminate
                flag = 'Terminate due to "output_t = NaN"'
                print('T_terminate=',t)
                error =1
                break
            Xt.append(Xt_i)
        if (error ==1): break     #If there exist NaN in Fi; then break and terminate   
        ####### To store output
        if save_output==True:
            output.append(Xt)         #Store the output Xt

        #----------------------------------------------#
        #     Generate exogeneous_load if stoch= True  #
        #----------------------------------------------#
        if stoch == True:
            for e_index in range(len(edges)):
                e_head = edges['init_node'][e_index]
                e_tail = edges['term_node'][e_index]
                network[e_head][e_tail]['noise_rel'] = network_noise[e_head][e_tail]['noise_rel'][t]
                network[e_head][e_tail]['noise_abs'] = network_noise[e_head][e_tail]['noise_abs'][t]
#         #--------------------------------------------------#
#         #     Compute potential value for plotting purpose #
#         #--------------------------------------------------#     
        load(network,Xt)                               #Import the load at Xt to the network
        pot.append(potential(Xt,pairs,network,edges,mean_noise))  #Compute the rosenthal value of Xt
        
        #----------------------------------------#
        #        Update alpha and gamma          #
        #----------------------------------------#
        gamma_prev = gamma
        gamma = ((2*gamma_prev + gamma0) + np.sqrt( 4*gamma_prev * gamma0 + (gamma0**2) )   )/2
        alpha = gamma_prev/gamma
        
        #----------------------------------------#
        #         Compute Z_tilde              #
        #----------------------------------------#
        Z_tilde=[]
        for i in range(N):
            Z_tilde_i = []
            for p in range(len(paths[i])):
                Z_tilde_ip = alpha *  Xt[i][p] + (1-alpha) * Zt[i][p]
                Z_tilde_i.append(Z_tilde_ip)
            Z_tilde.append(Z_tilde_i)
        #----------------------------------------------------
        #      Updating the weight Y
        #---------------------------------------------------
        load(network,Z_tilde)                          #Import the load in network at Z_tilde
        compute_e_cost(network,edges)                  #Re-compute the corresponding edges-costs at Z_tilde
        for i in range(N):
            for p in range(len(paths[i])):
                Y[i][p] = Y[i][p] - (1-alpha)*gamma * p_cost(network,paths,i,p)   
    
    class solution:
        def __init__(self,output, pot, flag):
            self.output = output
            self.pot = pot
            self.flag= flag
    return solution(output,pot,flag)


## 5. Adaptive Exponential Weights Algorithm

In [13]:
def AdaEW(pairs,network,paths,edges,T, theta,network_noise,mean_noise,stoch = True, gamma1=0,save_output=False):     #Include the input paths (pre-computed to save computations)
    print('#----------------------------------','\n','Run AdaWeight algorithm with T = ', T, ', stochastic = ', stoch)
    # Checking conditions of inputs:
    if not (len(pairs)== len(paths)):
        print('Error: inconsistent dimensions of pairs and paths')
        return
    if (stoch==True ) and ((network.number_of_edges()!= network_noise.number_of_edges()) or (T != network_noise.graph['T'])):
        print('Error: inconsistent data in network_noise')
        return     
    N=len(pairs)
    #INITIALIZATIONS
    output = []
    pot = []
    error = 0
    flag = 'Normal execute'
    #Initialize stepsizes:
    psi = 0
    gamma = gamma1     # If gamma1=0, we have the same initial point (logit at Y=0) as other algorithms
    
    # Initialization 
     ## Create for a zeros vector
    zeros=[]
    for i in range(N):
        zeros.append(np.zeros(len(paths[i])))
    ### Initialize Y at zeros
    Y = copy.deepcopy(zeros)
    Y_half=copy.deepcopy(zeros)
    S=copy.deepcopy(zeros)
    for e_index in range(len(edges)):
        e_head = edges['init_node'][e_index]
        e_tail = edges['term_node'][e_index]
        network[e_head][e_tail]['noise_rel']=0
        network[e_head][e_tail]['noise_abs']=0
        network[e_head][e_tail]['exo_load']=0
                    #------------------------------------------#
                    #         START OF MAIN ALGORITHM          #
                    #------------------------------------------#
    for t in range(1,T+1):
        if (t % 500 ==1): print('t=',t)    #To check the algorithm is running
        if (t==T): print('T_terminal=',t)
        #-------------------------------------
        #       Computing Z , Z_bar
        #----------------------------------
        Z_bar = []
        for i in range(N):
            Z_bar_i = []
            for p in range(len(paths[i])):
                diff = np.sum(np.exp(gamma* np.array(Y[i]) - gamma* Y[i][p]))
                Zip =  float(pairs['demand'][i]) / float(diff)
                Z_bar_i.append( (2/ (t*(t+1))) * ( t* Zip + S[i][p]  ) )
            Z_bar.append(Z_bar_i)
        #----------------------------------------------
        #     Generate exogeneous_load if stoch= True
        #---------------------------------------------
        if stoch == True:
            for e_index in range(len(edges)):
                e_head = edges['init_node'][e_index]
                e_tail = edges['term_node'][e_index]
                network[e_head][e_tail]['noise_rel']= network_noise[e_head][e_tail]['noise_rel'][t]    #load the exo_load from network_noise to network
                network[e_head][e_tail]['noise_abs']= network_noise[e_head][e_tail]['noise_abs'][t]
                
        #------------------------------------------
        #        Cost at Z_bar and update Y_half
        #-------------------------------------------
        load(network,Z_bar)         # Import network with load at Z_bar
        compute_e_cost(network,edges)                  #Re-compute the corresponding edges-costs at Z_bar
        C_Z_bar=[]
        for i in range(N):
            C_Z_bar_i=[]
            for p in range(len(paths[i])):
                cost_Z_bar_ip = p_cost(network,paths,i,p) 
                C_Z_bar_i.append(cost_Z_bar_ip )
                Y_half[i][p] = Y[i][p] - (t * cost_Z_bar_ip )
            C_Z_bar.append(C_Z_bar_i)
                
        #---------------------------------------
        #        Computing Z_half and Xt
        #---------------------------------------
        Xt = []
        for i in range(N):
            Xt_i = []
            for p in range(len(paths[i])):
                diff = np.sum(np.exp(gamma* np.array(Y_half[i]) - gamma*Y_half[i][p]))
                Z_half_ip  = float(pairs['demand'][i])/ float(diff)
                Xt_ip = ((2/ (t*(t+1))) * (t* Z_half_ip + S[i][p]  ) )
                S[i][p] = S[i][p] + t* Z_half_ip
                Xt_i.append(Xt_ip)
            Xt.append(Xt_i)
        
    
        ##------------------------------
        ##   Store result
        #-------------------------------
        if save_output==True:
            output.append(Xt)
        
        #---------------------------------------------
        # Compute potential at Xt for plotting purpose
        #---------------------------------------------
        load(network,Xt)                               #Import the load at Xt to the network
        pot.append(potential(Xt,pairs,network,edges,mean_noise))  #Compute the rosenthal value of Xt

        #-------------------------------------
        #        Update Y and S, psi and gamma              
        #-------------------------------------
        compute_e_cost(network,edges)                  #Re-compute the corresponding edges-costs at Xt
        norm_infty = 0
        for i in range(N):
            for p in range(len(paths[i])):
                cost_Xt_i_p = p_cost(network,paths,i,p) 
                Y[i][p] = Y[i][p] - (t *  cost_Xt_i_p )
                if np.abs(cost_Xt_i_p - C_Z_bar[i][p]) > norm_infty: norm_infty= np.abs(cost_Xt_i_p - C_Z_bar[i][p])
        psi = psi + (t**2) * (norm_infty**2)
        gamma = 1 / (np.sqrt(theta + psi))
        
#         print('Z_bar = ', Z_bar, '\n','Xt = ', Xt, '\n','S=',S)
    class solution:
        def __init__(self,output, pot,  flag):
            self.output = output
            self.pot = pot
            self.flag= flag
            
    return solution(output,pot,flag)


## 7. NUMERICAL EXPERIMENTS

## 7.1. STATIONARY SETTING

In [14]:
#------------------------------------------------------------------------------------------
#         STOCHASTIC
#-----------------------------------------------------------------------------------------
#-----------------------------------
# Choose a regime
#-----------------------------------
regime = 'stationary'

#-----------------------------------
#  Choose the beta
#-----------------------------------
betas = [0.001]        # the parameter beta (smoothness approximation) of AcceleWeight
T=1000                 # the learning horizon
theta=100              # a parameter to control the initial stepsize of AdaWeight (can be set to 1).

#-----------------------------------------------------------
# Pre-generate and store random exo-load to data/network_name/noise.gpickle
#----------------------------------------------------------
if regime=='stationary': 
    Stoch = False
    network_noise = []
    mean_noise = 0
else: 
    Stoch=True

#----------------------------
# Run AcceEW algorithm
#-------------------------       
for beta in betas:
    folder = directory_sol + '/'+ regime+ '/N='+str(N)+', T='+str(T)+ ', beta = ' + str(beta)+ '/'
    os.makedirs(os.path.dirname(folder), exist_ok=True)        
    SolAcceEW = AcceEW(pairs,network,paths,edges,T,network_noise,mean_noise,beta,stoch = Stoch) 
    save_potential(folder, potAcceEW=SolAcceEW.pot)      #save the potential values

# #------------------------------------------------------------------------------------
beta_best = 0.001       #beta_best = an "address" for saving results of ExpWeight and AdaWeight. Normally, we save the results of ExpWEight and ADaWeight to the same folder of AcceleWeight (with beta parameter = beta_best)
folder = directory_sol + '/'+ regime+ '/N='+str(N)+', T='+str(T)+ ', beta = ' + str(beta_best)+ '/'
#-----------------------
# Run AdaEW algorithm
#-------------------------
SolAdaEW= AdaEW(pairs,network,paths,edges,T, theta,network_noise,mean_noise,stoch = Stoch, gamma1=0)  
save_potential(folder, potAdaEW=SolAdaEW.pot)      #save the potential values
#-----------------------
# Run EW algorithm
#-------------------------
SolEW= EW(pairs,network,paths,edges,T,network_noise,mean_noise,stoch = Stoch, fixed_stepsize=False, stepsize=1)
save_potential(folder, potEW=SolEW.pot)      #save the potential values

   


#---------------------------------- 
 Run AcceWeight algorithm with T =  1000 , stochastic =  False
t= 1


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: overflow encountered in exp
D:\Programs\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


t= 501
T_terminal= 1000
#---------------------------------- 
 Run AdaWeight algorithm with T =  1000 , stochastic =  False
t= 1


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:83: RuntimeWarning: overflow encountered in exp
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in exp


t= 501
T_terminal= 1000
#---------------------------------- 
 Run EW algorithm with T =  1000 , stochastic =  False
t= 1


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: overflow encountered in exp


t= 501
T_terminal= 1000


## 7.2. STOCHASTIC SETTING

In [15]:
#------------------------------------------------------------------------------------------
#         STOCHASTIC
#-----------------------------------------------------------------------------------------
#-----------------------------------
# Choose a regime
#-----------------------------------
regime = 'stochastic'
instances = np.arange(1)        #Choose the set of indices to name the instances

beta = 1      # Smoothness approximation parameter of AcceleWeight
T=1000        # Learning horizon
theta=1000     # Controling the first step-size of AdaWeight

for instance in instances:
#-----------------------------------------------------------
# Pre-generate and store random exo-load to data/network_name/noise.gpickle
#----------------------------------------------------------
    if regime=='stationary': 
        Stoch = False
        network_noise = []
        mean_noise = 0
    else: 
        Stoch=True
        bound_noise = 0.1 # Determine the range of noise ---> generate noise by Uniform[-bound_noise, bound_noise]
        StdDev=5
        network_noise = generate_noise(network,T,bound_noise_rel=bound_noise,StdDevNoise_abs=StdDev)
        mean_noise = 0
        nx.write_gpickle(network_noise,directory+'noise,N='+str(N)+', T='+ str(T)+ ', instance ='+ str(instance)+'.gpickle')   

    folder = directory_sol + '/'+ regime+ '/N='+str(N)+', T='+str(T)+ ', beta = ' + str(beta)+', instance =' +str(instance)+ '/'
    os.makedirs(os.path.dirname(folder), exist_ok=True)        
    #-----------------------
    # Run AdaEW algorithm
    #-------------------------
    SolAdaEW= AdaEW(pairs,network,paths,edges,T, theta,network_noise,mean_noise,stoch = Stoch, gamma1=0)  
    save_potential(folder, potAdaEW=SolAdaEW.pot)      #save the potential values
    #----------------------------
    # Run AcceEW algorithm
    #-------------------------   
    SolAcceEW = AcceEW(pairs,network,paths,edges,T,network_noise,mean_noise,beta,stoch = Stoch) 
    save_potential(folder, potAcceEW=SolAcceEW.pot)      #save the potential values
    #-----------------------
    # Run EW algorithm
    #-------------------------
    SolEW= EW(pairs,network,paths,edges,T,network_noise,mean_noise,stoch = Stoch, fixed_stepsize=False, stepsize=1)
    save_potential(folder, potEW=SolEW.pot)      #save the potential values

#---------------------------------- 
 Run AdaWeight algorithm with T =  1000 , stochastic =  True
t= 1


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:83: RuntimeWarning: overflow encountered in exp
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in exp


t= 501
T_terminal= 1000
#---------------------------------- 
 Run AcceWeight algorithm with T =  1000 , stochastic =  True
t= 1
t= 501


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: overflow encountered in exp


T_terminal= 1000
#---------------------------------- 
 Run EW algorithm with T =  1000 , stochastic =  True
t= 1


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: overflow encountered in exp


t= 501
T_terminal= 1000
